# Import

In [ ]:
import pandas as pd, numpy as np
import plotly.express as px
from plotly.graph_objs import Figure
from pathlib import Path
from importlib import reload
import software.analysis as a
reload(a)

## Import CSV

In [ ]:
df = pd.read_csv("20240120_aggregated_all_stats_bootstraps100_stride10")

# Some of the Core Functions and Variables

In [ ]:
color="Version"
sort_col_name="Sorted_By"
date_title = "Date for Inclusion of Reference Structures"
good = 2
frac_title=f"Fraction of Poses < {good}Å from Reference"

In [ ]:
def rename_legend_labels(newname_dict, fig):
    fig.for_each_trace(lambda t: t.update(name = newname_dict[t.name],
                                          legendgroup = newname_dict[t.name],
                                          hovertemplate = t.hovertemplate.replace(t.name, newname_dict[t.name])
                                         ))
    return fig

In [ ]:
def replace_xaxis_labels(fig: Figure, axis_title):
    fig.for_each_xaxis(lambda x: x.update(title = ''))
    fig.add_annotation(x=0.5,y=-0.15,
                   text=axis_title, textangle=0,
                       font=dict(size=16),
                    xref="paper", yref="paper",
            showarrow=False,)
    return fig

In [ ]:
def replace_yaxis_labels(fig: Figure, axis_title):
    fig.for_each_yaxis(lambda y: y.update(title = ''))
    fig.add_annotation(x=-0.05,y=0.5,
                   text=axis_title, textangle=-90,
                       font=dict(size=16),
                    xref="paper", yref="paper",
            showarrow=False,)
    return fig

In [ ]:
def clean_labels(fig):
    fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[-1]))
    return fig

In [ ]:
def scatter_wrapper(df, kwarg_dict, 
                    x_axis_title=None, 
                    y_axis_title=None, 
                    replace_xaxis=False,
                    replace_y_axis=False,
                    clean=True,
                    x_axis_reversed=False
                   ):
    fig:Figure = px.scatter(df, **kwarg_dict, hover_data=df.columns)
    if x_axis_title:
        if replace_xaxis:
            fig = replace_xaxis_labels(fig, x_axis_title)
        else:
            fig.update_xaxes(title=x_axis_title)
    
    if y_axis_title:
        if replace_y_axis:
            fig = replace_yaxis_labels(fig, y_axis_title)
        else:
            fig.update_yaxes(title=y_axis_title)
    
    if clean:
        fig = clean_labels(fig)
    if x_axis_reversed:
        fig.update_xaxes(autorange="reversed")
    return fig

In [ ]:
basic_plot_kwargs = dict(color=color, 
                         )

In [ ]:
big_plot_kwargs = dict(facet_col=sort_col_name,
                         facet_row="Split", 
                         height=600, 
                         width=1200, )

In [ ]:
single_plot_kwargs = dict(height=400, width=600)

In [ ]:
tc_plot_kwargs = dict(x=tc,  
                      labels={tc: tc_title},
                     range_x=[-0.1,2.1],)

In [ ]:
fraction_plot_kwargs = dict(range_y=[-0.1,1.1])

In [ ]:
stats_kwargs = dict(y="Value", error_y="STD")

## Hybrid Only Plot

In [ ]:
from plotly.

In [ ]:
fig = scatter_wrapper(df[df["Version"] == "Hybrid-Only"], 
                      dict(
                          y="Fraction", color="Structure_Split", #facet_col="Version",
                          category_orders={"Structure_Split":["Random", "Structure_Date", "TanimotoCombo", "TanimotoCombo_R"]},
                          color_discrete_sequence=["#5ba300","#89ce00", "#0073e6","#e6308a","#b51963",],
                          error_y="Max", 
                          error_y_minus="Min",
                          template="plotly_white",
                           # **fraction_plot_kwargs,
                           x="Number of References", 
                          height=600,
                          width=800,
                           # **single_plot_kwargs
                          
                      ),
                     y_axis_title=frac_title,
                     x_axis_title="Number of References",
                     )
newlabels = {"random":"Random",
                                  "TanimotoCombo": "Increasing Chemical Similarity", 
                                  "TanimotoCombo_R": "Decreasing Chemical Similarity",
                                  "Structure_Date": "Date of Structure Deposition", }
fig = rename_legend_labels(newlabels, fig)
fig.update_layout(legend=dict(title="Structure Selection", 
                              x=0.5, y=0.1, 
                              traceorder='reversed', 
                              title_font_size=24, 
                              font_size=18, 
                              font_color='black'),
                 xaxis=dict(title_font=dict(size=24), color='black', tickfont=dict(size=18)), yaxis=dict(title_font=dict(size=24), color='black', tickfont=dict(size=18)))
fig.update_yaxes(tickvals=np.arange(0, 1.1, 0.1)) 
fig.show()
fig.write_image("20240120_structure_splits_hybrid.svg")

In [ ]:
import plotly
plotly.graph_objs.layout.Template()

In [ ]:
list(np.range(0,1,20))

In [ ]:
np.arange(0, 1.1, 0.05)